Calculates JJA zonal mean jet latitude values for 1979-2014 and 2070-2099 for CMIP5 models.  Outputs both the jet latitude of the ensemble mean zonal mean zonal wind and the jet latitude for the first ensemle member. Concatenates 1979-2005 of the historical simulations with 2005-2014 of the RCP8.5 simulations and uses 2070-2099 of the RCP8.5 simulations.
This code works on a CMIP archive that is internal to the Climate and Global Dynamics Laboratory, NCAR.  If you want to run this on your own archive, you'd need to modify histpath and rcp85path and make sure the directory structure is set up in the right way.

Output is saved to ecpaper2020/DATASORT/JLAT/DATA/jlatcmip5.nc

In [1]:
import importlib
import pandas as pd 
import xarray as xr
import numpy as np
from numpy import nan
import sys
import warnings

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import jlat_utils as jlat
from ecpaper_utils import calendar_utils as cal

importlib.reload(read)
importlib.reload(jlat)
importlib.reload(cal)

warnings.filterwarnings('ignore')

Set paths for CMIP5 models (historical and RCP8.5) and variable to use and the pressure level to use (in Pa)

In [2]:
histpath="/project/cmip5/historical/Amon/"
rcp85path="/project/cmip5/rcp85/Amon/"
var="ua"
plevuse="70000"
pathout="./DATA/"

Information on the models being used is provided in cmip5csvinfo.csv.  This contains information on the models, number of members and whether there's any special order for the member numbers i.e., if they don't simply go from 1 to N.  Read in this info and set up the dates for each period.

In [4]:
cmip5models=pd.read_csv('../cmip5csvinfo.csv')

ybegp = 1979 ; monbegp = 1 ; yendp = 2014 ; monendp = 12 # dates for Past period
ybegf = 2070 ; monbegf = 1 ; yendf = 2099 ; monendf = 12 # dates for Future period

# total number of months (used for checking)
nmonthsp = (yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf = (yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

# set up date names
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)

This is the main part of the script.  It loops over models and ensemble members and calculates the zonal mean zonal wind at 700hPa for JJA.  This is then interpolated onto a 1degree latitude grid and the jet latitude is calculated for both the ensemble mean and the first member.  These are then output to netcdf.

In [6]:
models=cmip5models['Model']
nmodels=models.size

# initialize member 1 and ensemble mean jet latitude arrays
jlat1memp = xr.DataArray(np.zeros(nmodels), coords=[models], dims="model", name="jlat1memp")
jlat1memf = xr.DataArray(np.zeros(nmodels), coords=[models], dims="model", name="jlat1memf")
jlatemp = xr.DataArray(np.zeros(nmodels), coords=[models], dims="model", name="jlatemp")
jlatemf = xr.DataArray(np.zeros(nmodels), coords=[models], dims="model", name="jlatemf")


# loop over models
for index, modname in models.iteritems():
    
    # --- sort out the PAST period ---
    nmems=cmip5models.loc[index, "Nmempast"]
    
    for imem in range(1, nmems+1, 1): 
        memstr="r"+str(imem)+"i1p1"
        # check if a special order is needed
        changeorder=cmip5models.loc[index,"specialorderpast"]
        if (type(changeorder) == str):
            changeordernp=np.array(changeorder.split(","))
            memstr="r"+str(changeordernp[imem-1])+"i1p1"
            
        print("Processing past for "+modname+" "+memstr+"...")
        histdir=histpath+var+"/"+modname+"/"+memstr+"/"
        rcp85dir=rcp85path+var+"/"+modname+"/"+memstr+"/"
        
        # read in zonal mean u, concatentate historical to 2005 and rcp8.5 to 2014
        uhist=read.read_zonalmean_1lev(histdir+"*.nc", datebegp, "2005-12", plevuse)
        urcp=read.read_zonalmean_1lev(rcp85dir+"*.nc", "2006-01", dateendp, plevuse)
        u=xr.concat([uhist, urcp], dim="time", join="override") 
        
        # check the array length.  If this fails, try ending historical in 2005-11
        # and beginning rcp in 2005-12 to fix Hadley center.
        if (u.time.size != nmonthsp):
            print("switching to the assumption that historical ends at 2005-11")
            uhist=read.read_zonalmean_1lev(histdir+"*.nc", datebegp, "2005-11", plevuse)
            urcp=read.read_zonalmean_1lev(rcp85dir+"*.nc", "2005-12", dateendp, plevuse)
            u=xr.concat([uhist, urcp], dim="time", join="override")
            
        # check again
        if (u.time.size != nmonthsp):
            print("something's wring, nmonthsp="+str(nmonthsp)+" but u has size "+str(u.time.size))
            
        # calculate JJA mean and interpolate
        ujja = cal.season_mean(u, "ua", season="JJA")
        ujja = ujja.dropna('lat')
        ujjainterp = ujja.interp(lat=np.linspace(-90,90,181), method="cubic")
        
        # calculate the jet latitude for 1 member.  Initialize ensemble mean
        if (imem == 1):
            uem = xr.DataArray(np.zeros(ujjainterp["lat"].size), dims=["lat"], coords=[ujjainterp["lat"]])
            jlatv, jspeedv = jlat.calcjetlat(ujjainterp, -80, -20)
            jlat1memp[index] = jlatv
            
        uem[:] = uem[:] + ujjainterp[:]/float(nmems) # calculate ensemble mean
        
    jlatv, jspeed = jlat.calcjetlat( uem, -80, -20)
    jlatemp[index] = jlatv

    # --- end sort out the past ---
    
    # --- sort out the future ---
    nmems=cmip5models.loc[index, "Nmemfuture"]
    for imem in range(1, nmems+1, 1): # loop over members
        memstr="r"+str(imem)+"i1p1"
        
        #check if a special order is needed
        changeorder=cmip5models.loc[index,"specialorderfuture"]
        if (type(changeorder) == str):
            changeordernp = np.array(changeorder.split(","))
            memstr="r"+str(changeordernp[imem-1])+"i1p1"
            
        print("Processing future for "+modname+" "+memstr+"...")
        rcp85di = rcp85path+var+"/"+modname+"/"+memstr+"/"
        u=read.read_zonalmean_1lev(rcp85dir+"*.nc", datebegf,dateendf,plevuse)
        
        # check the size
        if (u.time.size != nmonthsf):
            if (modname=="HadGEM2-ES"):
                u=u.isel(time=slice(0, u.time.size-1)) # taking care of another Hadley center problem
            if (u.time.size !=  nmonthsf):
                print("something's wrong, nmonthsf="+str(nmonthsf)+" but u has size "+str(u.time.size))
                sys.exit("u for future for "+modname+" "+memstr+" doesn't have the right size")
        
        ujja = cal.season_mean(u, "ua", season="JJA") # calculate the JJA mean
        ujja = ujja.dropna('lat')
        ujjainterp = ujja.interp(lat=np.linspace(-90,90,181), method="cubic")
        
        # calculate the jet latitude for 1 member.  Initialize ensemble mean
        if (imem == 1):
            uem = xr.DataArray(np.zeros(ujjainterp["lat"].size), dims=["lat"], coords=[ujjainterp["lat"]])
            jlatv, jspeedv = jlat.calcjetlat(ujjainterp, -80, -20)
            jlat1memf[index]=jlatv
            
        uem[:]=uem[:]+ujjainterp[:]/float(nmems) # calculate ensembl emean
    
    jlatv, jspeedv = jlat.calcjetlat(uem, -80, -20)
    jlatemf[index] = jlatv
    # --- end sort out the future ----
                               
# output to netcdf 
    
jlatemp.to_netcdf(path=pathout+"jlatcmip5.nc")
jlatemf.to_netcdf(path=pathout+"jlatcmip5.nc", mode="a")
jlat1memp.to_netcdf(path=pathout+"jlatcmip5.nc", mode="a")
jlat1memf.to_netcdf(path=pathout+"jlatcmip5.nc", mode="a")
    
print("*********************DONE***********************")
    

Processing past for ACCESS1-0 r1i1p1...
Processing future for ACCESS1-0 r1i1p1...
Processing past for ACCESS1-3 r1i1p1...
Processing future for ACCESS1-3 r1i1p1...
Processing past for bcc-csm1-1 r1i1p1...
Processing future for bcc-csm1-1 r1i1p1...
Processing past for bcc-csm1-1-m r1i1p1...
Processing future for bcc-csm1-1-m r1i1p1...
Processing past for BNU-ESM r1i1p1...
Processing future for BNU-ESM r1i1p1...
Processing past for CanESM2 r1i1p1...
Processing past for CanESM2 r2i1p1...
Processing past for CanESM2 r3i1p1...
Processing past for CanESM2 r4i1p1...
Processing past for CanESM2 r5i1p1...
Processing future for CanESM2 r1i1p1...
Processing future for CanESM2 r2i1p1...
Processing future for CanESM2 r3i1p1...
Processing future for CanESM2 r4i1p1...
Processing future for CanESM2 r5i1p1...
Processing past for CCSM4 r1i1p1...
Processing past for CCSM4 r2i1p1...
Processing past for CCSM4 r3i1p1...
Processing past for CCSM4 r4i1p1...
Processing past for CCSM4 r5i1p1...
Processing past 

In [14]:
print(ujja)

<xarray.DataArray 'ua' (lat: 144)>
dask.array<mean_agg-aggregate, shape=(144,), dtype=float32, chunksize=(144,), chunktype=numpy.ndarray>
Coordinates:
    plev     float64 7e+04
  * lat      (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
